In [1]:
!pip install transformers torch


In [2]:
from transformers import BertTokenizer, BertModel

# Charger le tokenizer et le modèle "recobo/agriculture-bert-uncased"
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import pandas as pd

# Chemin vers le fichier Excel du corpus
chemin_corpus_pertinent = "/content/drive/MyDrive/corpus pertinent.xlsx"

# Charger le corpus depuis le fichier Excel
corpus_pertinent_df = pd.read_excel(chemin_corpus_pertinent)

# Extraire la colonne contenant les textes de votre DataFrame (par exemple, 'Texte' comme nom de colonne)
corpus_texts = corpus_pertinent_df['Texte'].tolist()  # Convertir en liste de textes

# Afficher quelques textes pour vérifier
print(corpus_texts[:5])  # Afficher les 5 premiers textes


['Spiruline de Nayalgué : Le Burkina à l’ère de la production industrielle.lundi 31 mai 2010 à 03h54min.Le Premier ministre, Tertius Zongo a présidé, le vendredi 28 mai 2010, à Nayalgué, localité située à quelques kilomètres de Koudougou, l’inauguration d’une unité de production de spiruline. Il en a profité pour autoriser la mise officielle sur le marché du FACA, un médicament pour le traitement de la drépanocytose.  Réalisée par le projet Spiruline Nayalgué, l’unité de production qui vient d’être inaugurée est le fruit d’un partenariat réussi entre le diocèse de Koudougou, l’ONG Technap (France) et le gouvernement du Burkina Faso.  Entièrement moderne, l’unité comprend un bâtiment technique, des bureaux et laboratoires équipés, un champ solaire de 128 plaques, un local technique, 2400 m2 de bassins aménagés en modules de production, un château-d’eau, deux puits busés, une route d’accès, un parking auto/moto et deux allées pavées.  Pour le ministre de la Santé, Seydou Bouda, cette uni

In [4]:
from transformers import BertTokenizer, BertModel

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

# Tokeniser les textes
inputs_agri_bert = agri_bert_tokenizer(corpus_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Afficher les inputs tokenisés pour vérifier
print(inputs_agri_bert)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[  102, 10481,  2751,  ..., 12231,   205,   103],
        [  102,  4765, 30107,  ..., 27018, 30113,   103],
        [  102,  2277, 15430,  ...,   268,  2016,   103],
        ...,
        [  102,   493,   579,  ..., 25955, 12978,   103],
        [  102, 10053,  4564,  ...,  2300, 23070,   103],
        [  102,  1665,  4652,  ..., 13131,  6969,   103]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [5]:
import nltk
from nltk.corpus import stopwords
import re

# Télécharger les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Fonction pour nettoyer et supprimer les stopwords
def preprocess_text(text):
    # Nettoyer le texte (par exemple, supprimer la ponctuation et convertir en minuscules)
    text = re.sub(r'[^\w\s]', '', text.lower())

    # Supprimer les stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Appliquer le prétraitement à votre corpus
corpus_texts_cleaned = [preprocess_text(text) for text in corpus_texts]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# Tokeniser les textes prétraités
inputs_agri_bert = agri_bert_tokenizer(corpus_texts_cleaned, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Afficher les inputs tokenisés pour vérifier
print(inputs_agri_bert)


{'input_ids': tensor([[  102, 10481,  2751,  ..., 18116,   223,   103],
        [  102,  4765, 30107,  ...,   223,  3210,   103],
        [  102,  2277, 15430,  ...,   365,  4913,   103],
        ...,
        [  102,   493,  8451,  ...,  6820,   805,   103],
        [  102, 10053,  4564,  ..., 30110, 18844,   103],
        [  102,  1665,  4652,  ...,  5440,  9287,   103]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [7]:
!pip install torch


In [8]:
import torch


In [9]:
# Obtenir les embeddings avec le modèle Agriculture-BERT
with torch.no_grad():
    outputs_agri_bert = agri_bert_model(**inputs_agri_bert)

# Extraire les embeddings (les derniers états cachés du modèle)
embeddings = outputs_agri_bert.last_hidden_state

# Afficher la forme des embeddings pour vérifier
print(embeddings.shape)


torch.Size([43, 512, 768])


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Obtenir les embeddings des tokens pour un document donné (par exemple, le premier document)
embeddings_document = embeddings[0]

# Calculer la similarité cosinus entre les embeddings des tokens du document
similarity_matrix = cosine_similarity(embeddings_document.numpy())

# Obtenir les tokens correspondants
tokens = agri_bert_tokenizer.convert_ids_to_tokens(inputs_agri_bert['input_ids'][0])

# Afficher les tokens et leur similarité
for i, token in enumerate(tokens):
    print(f"Token: {token}, Similarité: {np.mean(similarity_matrix[i])}")


Token: [CLS], Similarité: 0.1788916438817978
Token: spir, Similarité: 0.28001898527145386
Token: ##ulin, Similarité: 0.2860215902328491
Token: ##e, Similarité: 0.39194804430007935
Token: de, Similarité: 0.37659335136413574
Token: na, Similarité: 0.2830219268798828
Token: ##yal, Similarité: 0.2652590274810791
Token: ##gue, Similarité: 0.3761301040649414
Token: le, Similarité: 0.3438965082168579
Token: burk, Similarité: 0.35866445302963257
Token: ##ina, Similarité: 0.36189091205596924
Token: a, Similarité: 0.37974563241004944
Token: ler, Similarité: 0.375102162361145
Token: ##e, Similarité: 0.4116583466529846
Token: de, Similarité: 0.38228553533554077
Token: la, Similarité: 0.3558077812194824
Token: production, Similarité: 0.38110625743865967
Token: indust, Similarité: 0.33911991119384766
Token: ##rie, Similarité: 0.27938395738601685
Token: ##lle, Similarité: 0.38516151905059814
Token: ##lund, Similarité: 0.359596848487854
Token: ##i, Similarité: 0.4172932505607605
Token: 31, Similarité:

In [11]:
def reconstruct_tokens(tokens):
    reconstructed_tokens = []
    current_token = ""

    for token in tokens:
        if token.startswith("##"):
            # Si le token commence par '##', c'est une continuation du mot précédent
            current_token += token[2:]
        else:
            # Si on a accumulé un mot, on l'ajoute à la liste
            if current_token:
                reconstructed_tokens.append(current_token)
            current_token = token

    # Ajouter le dernier mot s'il y en a un
    if current_token:
        reconstructed_tokens.append(current_token)

    return reconstructed_tokens

# Reconstituer les tokens pour le premier document
reconstructed_tokens = reconstruct_tokens(tokens)

# Afficher les tokens reconstitués
print(reconstructed_tokens)


['[CLS]', 'spiruline', 'de', 'nayalgue', 'le', 'burkina', 'a', 'lere', 'de', 'la', 'production', 'industriellelundi', '31', 'mai', '2010', 'a', '03h54minle', 'premier', 'ministre', 'tertius', 'zongo', 'preside', 'le', 'vendredi', '28', 'mai', '2010', 'a', 'nayalgue', 'localite', 'situee', 'a', 'quelques', 'kilometres', 'de', 'koudougou', 'linauguration', 'dune', 'unite', 'de', 'production', 'de', 'spiruline', 'il', 'en', 'profite', 'pour', 'autoriser', 'la', 'mise', 'officielle', 'sur', 'le', 'marche', 'du', 'faca', 'un', 'medicament', 'pour', 'le', 'traitement', 'de', 'la', 'drepanocytose', 'realisee', 'par', 'le', 'projet', 'spiruline', 'nayalgue', 'lunite', 'de', 'production', 'qui', 'vient', 'detre', 'inauguree', 'est', 'le', 'fruit', 'dun', 'partenariat', 'reussi', 'entre', 'le', 'diocese', 'de', 'koudougou', 'long', 'technap', 'france', 'et', 'le', 'gouvernement', 'du', 'burkina', 'faso', 'entierement', 'moderne', 'lunite', 'comprend', 'un', 'batiment', 'technique', 'des', 'burea

In [12]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_fr = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import re

# Fonction améliorée pour supprimer les stopwords, dates, heures et gérer les formats irréguliers
def preprocess_text_advanced_v3(text):
    # Supprimer les dates au format DD/MM/YYYY, DD-MM-YYYY, YYYY-MM-DD, ou Mois Année
    date_pattern = r'\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2}|(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4}|\d{4})\b'
    text = re.sub(date_pattern, '', text)

    # Supprimer les heures au format HH:MM, HHhMM, ou HHh
    time_pattern = r'\b\d{1,2}[:hH]\d{2}|\d{1,2}h\b'
    text = re.sub(time_pattern, '', text)

    # Supprimer les combinaisons de jour de la semaine avec dates (ex : 'lundi 31')
    day_date_pattern = r'\b(lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche)\s+\d{1,2}\b'
    text = re.sub(day_date_pattern, '', text)

    # Nettoyer le texte (par exemple, supprimer la ponctuation et convertir en minuscules)
    text = re.sub(r'[^\w\s]', '', text.lower())

    # Supprimer les stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words_fr]

    return ' '.join(tokens)

# Appliquer le prétraitement avancé à votre corpus
corpus_texts_cleaned_advanced_v3 = [preprocess_text_advanced_v3(text) for text in corpus_texts]

# Afficher quelques exemples après le prétraitement
print(corpus_texts_cleaned_advanced_v3[:5])


['spiruline nayalgué burkina lère production industrielle minle premier ministre tertius zongo a présidé nayalgué localité située quelques kilomètres koudougou linauguration dune unité production spiruline a profité autoriser mise officielle marché faca médicament traitement drépanocytose réalisée projet spiruline nayalgué lunité production vient dêtre inaugurée fruit dun partenariat réussi entre diocèse koudougou long technap france gouvernement burkina faso entièrement moderne lunité comprend bâtiment technique bureaux laboratoires équipés champ solaire 128 plaques local technique m2 bassins aménagés modules production châteaudeau deux puits busés route daccès parking automoto deux allées pavées ministre santé seydou bouda cette unité cache bien prouesses industrielles effet champ solaire exemple na encore dégal afrique louest produit actuellement heures pointe suffisamment délectricité alimenter toute lunité climatisation outils production compris excédant directement rétrocédé sona

In [14]:
corpus_texts_cleaned_advanced_v3

['spiruline nayalgué burkina lère production industrielle minle premier ministre tertius zongo a présidé nayalgué localité située quelques kilomètres koudougou linauguration dune unité production spiruline a profité autoriser mise officielle marché faca médicament traitement drépanocytose réalisée projet spiruline nayalgué lunité production vient dêtre inaugurée fruit dun partenariat réussi entre diocèse koudougou long technap france gouvernement burkina faso entièrement moderne lunité comprend bâtiment technique bureaux laboratoires équipés champ solaire 128 plaques local technique m2 bassins aménagés modules production châteaudeau deux puits busés route daccès parking automoto deux allées pavées ministre santé seydou bouda cette unité cache bien prouesses industrielles effet champ solaire exemple na encore dégal afrique louest produit actuellement heures pointe suffisamment délectricité alimenter toute lunité climatisation outils production compris excédant directement rétrocédé sona

In [15]:
import torch
from transformers import BertTokenizer, BertModel

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

# Fonction pour extraire les termes pertinents avec Agriculture-BERT
def extract_pertinent_terms(text, model, tokenizer, top_n=5):
    # Tokeniser le texte
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Obtenir les embeddings du modèle BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Embeddings des tokens
    token_embeddings = outputs.last_hidden_state.squeeze(0)

    # Calculer les similarités cosinus entre les tokens (par exemple, avec le premier token [CLS])
    cls_embedding = token_embeddings[0]  # Le premier token est [CLS], qui représente le texte complet
    similarities = torch.nn.functional.cosine_similarity(cls_embedding.unsqueeze(0), token_embeddings, dim=-1)

    # Obtenir les tokens originaux
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze().tolist())

    # Associer les tokens avec leur similarité
    token_similarity_pairs = [(token, similarity.item()) for token, similarity in zip(tokens, similarities)]

    # Trier les tokens par similarité (du plus pertinent au moins pertinent)
    token_similarity_pairs = sorted(token_similarity_pairs, key=lambda x: x[1], reverse=True)

    # Filtrer les sous-mots (tokens qui commencent par ##) et ne garder que les top_n termes
    pertinent_terms = [token for token, sim in token_similarity_pairs if not token.startswith('##')][:top_n]

    return pertinent_terms

# Appliquer l'extraction des termes pertinents sur le corpus prétraité
corpus_pertinent_terms = [extract_pertinent_terms(text, agri_bert_model, agri_bert_tokenizer) for text in corpus_texts_cleaned_advanced_v3]

# Afficher les termes pertinents pour les 5 premiers textes
for i, terms in enumerate(corpus_pertinent_terms[:5]):
    print(f"Texte {i+1}: Termes pertinents: {terms}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Texte 1: Termes pertinents: ['[CLS]', '[SEP]', 'burk', 'entre', 'future']
Texte 2: Termes pertinents: ['[CLS]', '[SEP]', 'millions', 'occasion', 'lun']
Texte 3: Termes pertinents: ['[CLS]', '[SEP]', 'a', 'spark', 'look']
Texte 4: Termes pertinents: ['[CLS]', '[SEP]', 'sect', 'identifies', 'plus']
Texte 5: Termes pertinents: ['[CLS]', '[SEP]', 'site', 'sur', 'excision']


In [16]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

def extract_terms_and_scores(text, model, tokenizer):
    # Tokeniser le texte
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Obtenir les embeddings du modèle BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Embeddings des tokens
    token_embeddings = outputs.last_hidden_state.squeeze(0)

    # Calculer les similarités cosinus entre les tokens et le token [CLS]
    cls_embedding = token_embeddings[0]
    similarities = torch.nn.functional.cosine_similarity(cls_embedding.unsqueeze(0), token_embeddings, dim=-1)

    # Créer une liste de tuples (token, score)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    terms_and_scores = [(token, score.item()) for token, score in zip(tokens, similarities)]

    return terms_and_scores

# Appliquer la fonction à chaque texte du corpus
all_terms_and_scores = []
for text in corpus_texts_cleaned_advanced_v3:
    all_terms_and_scores.extend(extract_terms_and_scores(text, agri_bert_model, agri_bert_tokenizer))

# Créer un DataFrame pandas
df = pd.DataFrame(all_terms_and_scores, columns=['Token', 'Score'])

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores.xlsx", index=False)

Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# ... (Code pour charger le modèle et le tokenizer)

def extract_terms_and_scores(text, model, tokenizer):
  # ... (Code pour obtenir les embeddings et les similarités)

  # Reconstruire les mots à partir des tokens
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  reconstructed_tokens = []
  current_word = ""
  for token in tokens:
    if token.startswith("##"):
      current_word += token[2:]
    else:
      if current_word:
        reconstructed_tokens.append(current_word)
      current_word = token
  if current_word:
    reconstructed_tokens.append(current_word)

  # Créer une liste de tuples (mot, score)
  terms_and_scores = [(word, score.item()) for word, score in zip(reconstructed_tokens, similarities)]
  return terms_and_scores

# ... (Reste du code pour créer et afficher le DataFrame)

In [18]:
# Créer un DataFrame pandas
df1 = pd.DataFrame(all_terms_and_scores, columns=['Mots', 'Score'])

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores.xlsx", index=False)

In [19]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

def extract_terms_and_scores(text, model, tokenizer):
  # Tokeniser le texte
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

  # Obtenir les embeddings du modèle BERT
  with torch.no_grad():
    outputs = model(**inputs)

  # Embeddings des tokens
  token_embeddings = outputs.last_hidden_state.squeeze(0)

  # Calculer les similarités cosinus entre les tokens et le token [CLS]
  cls_embedding = token_embeddings[0]
  similarities = torch.nn.functional.cosine_similarity(cls_embedding.unsqueeze(0), token_embeddings, dim=-1)

  # Reconstruire les mots à partir des tokens
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  reconstructed_tokens = []
  current_word = ""
  for token in tokens:
    if token.startswith("##"):
      current_word += token[2:]
    else:
      if current_word:
        reconstructed_tokens.append(current_word)
      current_word = token
  if current_word:
    reconstructed_tokens.append(current_word)

  # Créer une liste de tuples (mot, score)
  terms_and_scores = [(word, score.item()) for word, score in zip(reconstructed_tokens, similarities)]
  return terms_and_scores

# Appliquer la fonction à chaque texte du corpus
all_terms_and_scores = []
for text in corpus_texts_cleaned_advanced_v3:
  all_terms_and_scores.extend(extract_terms_and_scores(text, agri_bert_model, agri_bert_tokenizer))

# Créer un DataFrame pandas
df = pd.DataFrame(all_terms_and_scores, columns=['Mot', 'Score'])

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores.xlsx", index=False)

# Afficher le DataFrame
df

Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Mot,Score
0,[CLS],1.000000
1,spiruline,0.084383
2,nayalgue,0.124054
3,burkina,0.217872
4,lere,0.064957
...,...,...
8509,14,0.183460
8510,difficiles,0.213480
8511,alimenter,0.093648
8512,parce,0.182889


In [20]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import re

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

def extract_terms_and_scores(text, model, tokenizer):
  # Tokeniser le texte
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

  # Obtenir les embeddings du modèle BERT
  with torch.no_grad():
    outputs = model(**inputs)

  # Embeddings des tokens
  token_embeddings = outputs.last_hidden_state.squeeze(0)

  # Calculer les similarités cosinus entre les tokens et le token [CLS]
  cls_embedding = token_embeddings[0]
  similarities = torch.nn.functional.cosine_similarity(cls_embedding.unsqueeze(0), token_embeddings, dim=-1)

  # Reconstruire les mots à partir des tokens
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  reconstructed_tokens = []
  current_word = ""
  for token in tokens:
    if token.startswith("##"):
      current_word += token[2:]
    else:
      if current_word:
        reconstructed_tokens.append(current_word)
      current_word = token
  if current_word:
    reconstructed_tokens.append(current_word)

  # Supprimer les tokens spéciaux et les chiffres
  terms_and_scores = [(word, score.item()) for word, score in zip(reconstructed_tokens, similarities) if word not in ['[CLS]', '[SEP]'] and not re.match(r'\b\d+\b', word)]

  return terms_and_scores

# Appliquer la fonction à chaque texte du corpus
all_terms_and_scores = []
for text in corpus_texts_cleaned_advanced_v3:
  all_terms_and_scores.extend(extract_terms_and_scores(text, agri_bert_model, agri_bert_tokenizer))

# Créer un DataFrame pandas
df = pd.DataFrame(all_terms_and_scores, columns=['Mot', 'Score'])

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores.xlsx", index=False)

# Afficher le DataFrame
df

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Mot,Score
0,spiruline,0.084383
1,nayalgue,0.124054
2,burkina,0.217872
3,lere,0.064957
4,production,0.045174
...,...,...
8142,partie,0.144184
8143,secteur,0.171050
8144,difficiles,0.213480
8145,alimenter,0.093648


In [21]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import re

# Charger le modèle Agriculture-BERT et le tokenizer
agri_bert_tokenizer = BertTokenizer.from_pretrained('recobo/agriculture-bert-uncased')
agri_bert_model = BertModel.from_pretrained('recobo/agriculture-bert-uncased')

def extract_terms_and_scores(text, model, tokenizer):
  # Tokeniser le texte
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

  # Obtenir les embeddings du modèle BERT
  with torch.no_grad():
    outputs = model(**inputs)

  # Embeddings des tokens
  token_embeddings = outputs.last_hidden_state.squeeze(0)

  # Calculer les similarités cosinus entre les tokens et le token [CLS]
  cls_embedding = token_embeddings[0]
  similarities = torch.nn.functional.cosine_similarity(cls_embedding.unsqueeze(0), token_embeddings, dim=-1)

  # Reconstruire les mots à partir des tokens
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  reconstructed_tokens = []
  current_word = ""
  for token in tokens:
    if token.startswith("##"):
      current_word += token[2:]
    else:
      if current_word:
        reconstructed_tokens.append(current_word)
      current_word = token
  if current_word:
    reconstructed_tokens.append(current_word)

  # Supprimer les tokens spéciaux, les chiffres et trier par score
  terms_and_scores = [(word, score.item()) for word, score in zip(reconstructed_tokens, similarities) if word not in ['[CLS]', '[SEP]'] and not re.match(r'\b\d+\b', word)]
  terms_and_scores.sort(key=lambda x: x[1], reverse=True) # Trier par score décroissant

  return terms_and_scores

# Appliquer la fonction à chaque texte du corpus
all_terms_and_scores = []
for text in corpus_texts_cleaned_advanced_v3:
  all_terms_and_scores.extend(extract_terms_and_scores(text, agri_bert_model, agri_bert_tokenizer))

# Créer un DataFrame pandas
df = pd.DataFrame(all_terms_and_scores, columns=['Mot', 'Score'])

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores_tries.xlsx", index=False)

# Afficher le DataFrame
df

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Mot,Score
0,champ,0.999912
1,pointe,0.322948
2,directement,0.301680
3,unite,0.295927
4,retrocede,0.290744
...,...,...
8142,1er,0.055218
8143,pourquoi,0.052078
8144,efficacite,0.052041
8145,cest,0.046949


In [22]:
def extract_terms_and_scores(text, model, tokenizer):
  # ... (le reste du code de la fonction reste inchangé)

  # Supprimer les tokens spéciaux, les chiffres et le mot "1er", puis trier par score
  terms_and_scores = [(word, score.item()) for word, score in zip(reconstructed_tokens, similarities) if word not in ['[CLS]', '[SEP]', "1er"] and not re.match(r'\b\d+\b', word)]
  terms_and_scores.sort(key=lambda x: x[1], reverse=True)

  return terms_and_scores

In [23]:
df

,Mot,Score
0,champ,0.999912
1,pointe,0.322948
2,directement,0.301680
3,unite,0.295927
4,retrocede,0.290744
...,...,...
8142,1er,0.055218
8143,pourquoi,0.052078
8144,efficacite,0.052041
8145,cest,0.046949


In [24]:
df.head(n=10)  # Afficher les 10 mots avec les scores les plus élevés


,Mot,Score
0,champ,0.999912
1,pointe,0.322948
2,directement,0.301680
3,unite,0.295927
4,retrocede,0.290744
5,buses,0.287730
6,seconde,0.276559
7,fiers,0.275285
8,techniciens,0.269134
9,pouvons,0.268893
